# Actividad 5.3 (Valores Nulos)
- Miranda Eugenia Colorado Arróniz A01737027

In [5]:
# Importar el CSV
import pandas as pd
df = pd.read_csv('hawaii.csv')

In [ ]:
# Ver los valores nulos por columna
df_reindexado.isnull().sum()

id                                                  0
listing_url                                         0
name                                                0
neighborhood_overview                           18697
picture_url                                         0
host_id                                             0
host_url                                            0
host_name                                           6
host_since                                          6
host_location                                    9434
host_about                                      11544
host_response_time                               2799
host_response_rate                               2799
host_acceptance_rate                             2187
host_is_superhost                                 674
host_total_listings_count                           6
host_verifications                                  6
host_identity_verified                              6
price                       

- **Variables numéricas continuas (ej. price, review_scores_rating, availability_365, etc.):**
  - Si la variable es simétrica y sin valores extremos, se usa la **media**.
  - Si la variable tiene sesgo o valores atípicos, se usa la **mediana**.
- **Variables categóricas o de texto (ej. host_name, host_response_time, neighborhood_overview, etc.):**
  - Se usa el string **'Sin dato'** para indicar ausencia de información.
- **Variables de fechas (ej. host_since, first_review, last_review):**
  - Se usa **propagación hacia adelante (ffill)**, ya que suelen tener sentido temporal.
- **Variables con muy alto porcentaje de nulos (>80%) o irrelevantes:**
  - Se pueden eliminar si no aportan valor o si la imputación no es confiable.

In [ ]:
df_imp = df_reindexado.copy()

for col in df_imp.columns:
    tipo = df_imp[col].dtype
    porc_nulos = df_imp[col].isnull().mean() * 100
    
    # Eliminar columna si tiene >80% nulos y es irrelevante
    if porc_nulos > 80 and col not in ['host_about', 'neighborhood_overview']:
        df_imp.drop(columns=col, inplace=True)
        continue
    
    # Fechas: ffill
    if 'date' in str(col).lower() or 'since' in str(col).lower() or 'review' in str(col).lower():
        df_imp[col] = df_imp[col].fillna(method='ffill')
        
    # Numéricas: mediana si hay outliers, si no media
    elif tipo in [np.float64, np.int64]:
        if df_imp[col].skew() > 1 or df_imp[col].skew() < -1:
            df_imp[col] = df_imp[col].fillna(df_imp[col].median())
        else:
            df_imp[col] = df_imp[col].fillna(df_imp[col].mean())
            
    # Categóricas: 'Sin dato'
    elif tipo == object:
        df_imp[col] = df_imp[col].fillna('Sin dato')
        
    # Otros casos: bfill como refuerzo
    else:
        df_imp[col] = df_imp[col].fillna(method='bfill')
        
df_imp.isnull().sum()

/var/folders/qg/1b6k32sd5_d2jr8q3bhrhg1r0000gn/T/ipykernel_6671/979345398.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_imp[col] = df_imp[col].fillna(method='ffill')
/var/folders/qg/1b6k32sd5_d2jr8q3bhrhg1r0000gn/T/ipykernel_6671/979345398.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_imp[col] = df_imp[col].fillna(method='ffill')


id                                              0
listing_url                                     0
name                                            0
neighborhood_overview                           0
picture_url                                     0
host_id                                         0
host_url                                        0
host_name                                       0
host_since                                      0
host_location                                   0
host_about                                      0
host_response_time                              0
host_response_rate                              0
host_acceptance_rate                            0
host_is_superhost                               0
host_total_listings_count                       0
host_verifications                              0
host_identity_verified                          0
price                                           0
minimum_nights                                  0


- **price, review_scores_rating, availability_365**: Se usó la mediana si había valores atípicos, si no la media.
- **host_name, host_response_time, host_location**: Se usó 'Sin dato' para indicar ausencia de información.
- **host_since, first_review, last_review**: Se usó ffill (propagación hacia adelante) por ser fechas.
- **host_about, neighborhood_overview**: No se eliminaron aunque tienen muchos nulos, por su posible valor descriptivo.
- **Otras columnas**: Se usó bfill si no entraban en los casos anteriores.

In [8]:
# Guardar en un nuevo archivo CSV
df_imp.to_csv('hawaii_sin_nulos.csv', index=False)